### Load the Libraries

In [274]:
from Bio import SeqIO
import re
# import math

In [1]:
# Primary libraries
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
# Secondary libraries
from tqdm import tqdm
# from scipy import stats
import scipy
# from glob import glob
import os
# from matplotlib import dates as mpl_dates
# import datetime
# from datetime import date
# import matplotlib.patches as mpatches
# from matplotlib import cm
# from colorspacious import cspace_converter
# from collections import OrderedDict
# import matplotlib.ticker as ticker

### Load the files

We are dealing with .gb files with single records each.

From (https://warwick.ac.uk/fac/sci/moac/people/students/peter_cock/python/genbank/)-

    Depending on the type of GenBank file(s) you are interested in, they will either contain a single record, or multiple records. You can easily determine this by looking at the raw file - each record will start with a LOCUS line, followed by various other header lines, usually a list of features, the sequence data, and ends with a // line (slash slash).

In [4]:
count = SeqIO.convert("./genbank_files/bacillus_subtilis.gb", "genbank",
                      "./converted_fasta_files/bacillus_subtilis.fasta", "fasta")
print(f'Converted {count} records')

Converted 1 records


In [157]:
# Loading the genbank file (given it has only one record; otherwise looping over it is required)

gb_record = SeqIO.read(open("./genbank_files/bacillus_subtilis.gb", 'r'), "genbank")
print(f'Name {gb_record.name}, {len(gb_record.features)} features')

Name NC_000964, 9074 features


In [10]:
print(gb_record.features[9000])

type: gene
location: [4183444:4183897](-)
qualifiers:
    Key: db_xref, Value: ['GeneID:937872']
    Key: gene, Value: ['yybA']
    Key: locus_tag, Value: ['BSU_40710']
    Key: old_locus_tag, Value: ['BSU40710']



In [23]:
print(gb_record.features[9000].strand)
print(gb_record.features[0].strand)

-1
1


In [21]:
l = []

for feature in gb_record.features:
    if feature.type not in l:
        l.append(feature.type)

print(l)

['source', 'gene', 'CDS', 'rRNA', 'tRNA', 'misc_RNA', 'misc_feature', 'ncRNA']


In [159]:
genes = []

for feature in gb_record.features:
    if feature.type == 'gene':
        genes.append(feature)

print(len(genes))

4536


In [162]:
print(type(genes[1].qualifiers['gene']))
print(genes[1].qualifiers['gene'])
print(genes[1].qualifiers['gene'][0])
print(genes[1].qualifiers['db_xref'][0][7:])

<class 'list'>
['dnaN']
dnaN
939970


Locations provided by **BioPython** is optimum for python purposes.
- start and end provided by it are 1938 and 3075 respectively (0-based indexing and it assumes that end position is not included),
- while in our actual file it is 1939 and 3075 (1-based indexing and it assumes that both start and end positions are included).

This way we can directly slice seq string using locations provided to obtain the seq for the features of our interest.

In [37]:
print(gb_record.features[3])

print(gb_record.features[3].location)

# print(gb_record.seq[gb_record.features[3].location]) # throws error
print(gb_record.seq[gb_record.features[3].location.nofuzzy_start:
                    gb_record.features[3].location.nofuzzy_end])

print('\n')
print(gb_record.seq[1938:3075])
print('\n')

# To obtain the DNA sequence corresponding to the complement, use .reverse_complement()
print(gb_record.seq[1938:3075].reverse_complement())

type: gene
location: [1938:3075](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:939970']
    Key: gene, Value: ['dnaN']
    Key: locus_tag, Value: ['BSU_00020']
    Key: old_locus_tag, Value: ['BSU00020']

[1938:3075](+)
ATGAAATTCACGATTCAAAAAGATCGTCTTGTTGAAAGTGTCCAAGATGTATTAAAAGCAGTTTCATCCAGAACCACGATTCCCATTCTGACTGGTATTAAAATTGTTGCATCAGATGATGGAGTATCCTTTACAGGGAGTGACTCAGATATTTCTATTGAATCCTTCATTCCAAAAGAAGAAGGAGATAAAGAAATCGTCACTATTGAACAGCCCGGAAGCATCGTTTTACAGGCTCGCTTTTTTAGTGAAATTGTAAAAAAATTGCCGATGGCAACTGTAGAAATTGAAGTCCAAAATCAGTATTTGACGATTATCCGTTCTGGTAAAGCTGAATTTAATCTAAACGGACTGGATGCTGATGAATATCCGCACTTGCCGCAGATTGAAGAGCATCATGCGATTCAGATCCCAACTGATTTGTTAAAAAATCTAATCAGACAAACAGTATTTGCAGTGTCCACCTCAGAAACACGCCCTATCTTGACAGGTGTAAACTGGAAAGTGGAGCAAAGTGAATTATTATGCACTGCAACGGATAGCCACCGTCTTGCATTAAGAAAGGCGAAACTTGATATTCCAGAAGACAGATCTTATAACGTCGTGATTCCGGGAAAAAGTTTAACTGAACTCAGCAAGATTTTAGATGACAACCAGGAACTTGTAGATATCGTCATCACAGAAACCCAAGTTCTGTTTAAAGCGAAAAACGTCTTGTTCTTCTCACGGCTTCTGGACGGGAATTATCCAGACACAACCAGCCTGATTCCGCAAGAC

In [39]:
print(gb_record.features[9000].strand)
print(gb_record.features[0].strand)
print(type(gb_record.features[9000].strand))
print(type(gb_record.features[0].strand))

-1
1
<class 'int'>
<class 'int'>


1. **The DDBJ/ENA/GenBank Feature Table Definition**: 
    Documentation of features in genbank files. Very good document, must go-through this once.
    
    Source:
    - (https://www.insdc.org/documents/feature_table.html)



2. **Locus tag:**
    Locus_tags are identifiers that are systematically applied to every gene in a genome.
    These tags have become surrogate gene names by the biological community. If two
    submitters of two different genomes use the same systematic names to describe two very
    different genes in two very different genomes, it can be very confusing. In order to
    prevent this from happening INSD has created a registry of locus_tag prefixes.
    Submitters of eukaryotic and prokaryotic genomes should register their prefix prior to
    submitting their genome. All components of a project (such as multiple chromosomes or
    plasmids, etc) should use the same locus_tag prefix. 
    
    Source:
    - (https://www.ncbi.nlm.nih.gov/genomes/locustag/Proposal.pdf)
    - (https://www.ncbi.nlm.nih.gov/genbank/genomesubmit_annotation/#locus_tag)

### Extracting desirable information

Useful videos for the analysis done later-
1. https://www.youtube.com/watch?v=LdQV3cbUwEE&list=PLe1-kjuYBZ05T9iHV_z60B9mpFt201ND5&index=8
2. https://www.youtube.com/watch?v=HP7ThAj_f1E

_Both videos are on Youtube @Bioinformatics Coach_

KeyError resolution -

    gene_name = gene.qualifiers['gene'][0]
    gene_name = gene.qualifiers.get('gene',['unavailable'])[0]

_Source_: https://bioinformatics.stackexchange.com/questions/15454/keyerror-when-getting-features-from-a-genbank-file-with-biopython-with-some-acce/15456#15456

In [307]:
# To clear the environment variables stored earlier in the notebook
del count, gb_record, l, genes, i

In [188]:
# # OLD working function for 200 bp only
# def genbank_file_reader(file_name):
#     """Takes in genbank file name in the folder ./genbank_files/;
#     Outputs the dataframe for that genbank file
#     """
#     gb_record = SeqIO.read(open(f"./genbank_files/{file_name}", 'r'), 'genbank')
#     print(f'Name {gb_record.name}, {len(gb_record.features)} features')
    
#     data = []

#     allgenes = (
#         feature
#         for feature in gb_record.features
#         if feature.type == 'gene'
#     )

#     for gene in allgenes:

#         gene_name = gene.qualifiers.get('gene',['unavailable'])[0]
#         gene_ID = gene.qualifiers['db_xref'][0][7:]
# #         gene_ID = gene.qualifiers.get('db_xref', ['GeneID:unavailable'])[0][7:]        
#         start_pos = gene.location.nofuzzy_start
#     #     start_pos = gene.location.nofuzzy_start + 1 # if 1-based indexing required
#         end_pos = gene.location.nofuzzy_end
#         strand_sense = gene.strand

#         gene_seq = gene.extract(gb_record).seq

#     #     For +ve sense strand
#         if strand_sense == 1:
#             if len(gb_record.seq[:start_pos]) >= 200:
#                 upstream_flank = gb_record.seq[start_pos-200:start_pos]
#             else:
#                 upstream_flank = gb_record.seq[:start_pos]

#             if len(gb_record.seq[end_pos:]) >= 200:
#                 downstream_flank = gb_record.seq[end_pos:end_pos+200]
#             else:
#                 downstream_flank = gb_record.seq[end_pos:]
#     #     For -ve sense strand
#         elif strand_sense == -1:
#             if len(gb_record.seq[:start_pos]) >= 200:
#                 downstream_flank = gb_record.seq[start_pos-200:start_pos].reverse_complement()
#             else:
#                 downstream_flank = gb_record.seq[:start_pos].reverse_complement()

#             if len(gb_record.seq[end_pos:]) >= 200:
#                 upstream_flank = gb_record.seq[end_pos:end_pos+200].reverse_complement()
#             else:
#                 upstream_flank = gb_record.seq[end_pos:].reverse_complement()

#         data.append((gene_ID, gene_name, start_pos, end_pos, strand_sense, 
#                      str(gene_seq), str(upstream_flank), str(downstream_flank)))    


#     # Creating an empty dataframe
#     df = pd.DataFrame(data, columns=['gene_ID', 
#                                      'gene_name', 
#                                      'start_position', 
#                                      'end_position', 
#                                      'strand_sense', 
#                                      'gene_seq', 
#                                      'upstream_flank', 
#                                      'downstream_flank'])
# # #     Saving the dataframe to a .csv file
#     df.to_csv('./csv_files/'+file_name[:-3]+'.csv', index=False)

In [312]:
def genbank_file_reader(file_name):
    """Takes in genbank file name in the folder ./genbank_files/;
    Outputs the dataframe for that genbank file
    """
    gb_record = SeqIO.read(open(f"./genbank_files/{file_name}", 'r'), 'genbank')
    print(f'Name {gb_record.name}, {len(gb_record.features)} features')
    
    data = []

    allgenes = (
        feature
        for feature in gb_record.features
        if feature.type == 'gene'
    )

    for gene in allgenes:

        gene_name = gene.qualifiers.get('gene',['unavailable'])[0]
        gene_ID = gene.qualifiers['db_xref'][0][7:]
#         gene_ID = gene.qualifiers.get('db_xref', ['GeneID:unavailable'])[0][7:]        
        start_pos = gene.location.nofuzzy_start
    #     start_pos = gene.location.nofuzzy_start + 1 # if 1-based indexing required
        end_pos = gene.location.nofuzzy_end
        strand_sense = gene.strand

        gene_seq = gene.extract(gb_record).seq

    #     For +ve sense strand
        if strand_sense == 1:
            if len(gb_record.seq[:start_pos]) >= 200:
                upstream_flank = gb_record.seq[start_pos-200:start_pos]
            else:
                upstream_flank = gb_record.seq[:start_pos]

            if len(gb_record.seq[end_pos:]) >= 200:
                downstream_flank = gb_record.seq[end_pos:end_pos+200]
            else:
                downstream_flank = gb_record.seq[end_pos:]
    #     For -ve sense strand
        elif strand_sense == -1:
            if len(gb_record.seq[:start_pos]) >= 200:
                downstream_flank = gb_record.seq[start_pos-200:start_pos].reverse_complement()
            else:
                downstream_flank = gb_record.seq[:start_pos].reverse_complement()

            if len(gb_record.seq[end_pos:]) >= 200:
                upstream_flank = gb_record.seq[end_pos:end_pos+200].reverse_complement()
            else:
                upstream_flank = gb_record.seq[end_pos:].reverse_complement()
    
    # Addition starts ---------------------------------------------------------------
    #     For +ve sense strand
        if strand_sense == 1:
            if len(gb_record.seq[:start_pos]) >= 203:
                upstream_flank_203 = gb_record.seq[start_pos-200:start_pos+3]
            else:
                upstream_flank_203 = gb_record.seq[:start_pos+3]

            if len(gb_record.seq[end_pos:]) >= 203:
                downstream_flank_203 = gb_record.seq[end_pos:end_pos+203]
            else:
                downstream_flank_203 = gb_record.seq[end_pos:]
    #     For -ve sense strand
        elif strand_sense == -1:
            if len(gb_record.seq[:start_pos]) >= 203:
                downstream_flank_203 = gb_record.seq[start_pos-203:start_pos].reverse_complement()
            else:
                downstream_flank_203 = gb_record.seq[:start_pos].reverse_complement()

            if len(gb_record.seq[end_pos:]) >= 203:
                upstream_flank_203 = gb_record.seq[end_pos-3:end_pos+200].reverse_complement()
            else:
                upstream_flank_203 = gb_record.seq[end_pos-3:].reverse_complement()                
    # Addition ends ---------------------------------------------------------------            
                
        data.append((gene_ID, gene_name, start_pos, end_pos, strand_sense, 
                     str(gene_seq), str(upstream_flank), str(downstream_flank), 
                     str(upstream_flank_203), str(downstream_flank_203)))    


    # Creating an empty dataframe
    df = pd.DataFrame(data, columns=['gene_ID', 
                                     'gene_name', 
                                     'start_position', 
                                     'end_position', 
                                     'strand_sense', 
                                     'gene_seq', 
                                     'upstream_flank', 
                                     'downstream_flank', 
                                     'upstream_flank_203', 
                                     'downstream_flank_203'])
# #     Saving the dataframe to a .csv file
    df.to_csv('./csv_files/'+file_name[:-3]+'.csv', index=False)

How to iterate over a given directory:
https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory

In [313]:
# To do the flank calculations for all files
directory = os.fsencode('./genbank_files/')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.gb'):
        print(filename)
        genbank_file_reader(filename)

bacillus_subtilis.gb
Name NC_000964, 9074 features
streptococcus_pneumoniae.gb
Name NZ_CP020549, 4328 features
klebsiella_pneumoniae.gb
Name NC_016845, 10894 features


#### Rough work

In [67]:
# # Loading the genbank file (given it has only one record; otherwise looping over it is required)

# gb_record = SeqIO.read(open("./genbank_files/bacillus_subtilis.gb", 'r'), 'genbank')
# print(f'Name {gb_record.name}, {len(gb_record.features)} features')

# # print(f'Total genes in this file are {len(allgenes)}')

# data = []

Name NC_000964, 9074 features


In [133]:
# allgenes = (
#     feature
#     for feature in gb_record.features
#     if feature.type == 'gene'
# )

# for gene in allgenes:
# #     gene_name = gene.qualifiers['gene'][0]
#     gene_name = gene.qualifiers.get('gene',['unavailable'])[0]
#     start_pos = gene.location.nofuzzy_start
# #     start_pos = gene.location.nofuzzy_start + 1 # if 1-based indexing required
#     end_pos = gene.location.nofuzzy_end
#     strand_sense = gene.strand
    
#     gene_seq = gene.extract(gb_record).seq
# #     print(gene_name, start_pos, end_pos, strand_sense)
# #     print(f'\n\n{gene_seq}')
    
# #     For +ve sense strand
#     if strand_sense == 1:
#         if len(gb_record.seq[:start_pos]) >= 200:
#             upstream_flank = gb_record.seq[start_pos-200:start_pos]
#         else:
#             upstream_flank = gb_record.seq[:start_pos]
        
#         if len(gb_record.seq[end_pos:]) >= 200:
#             downstream_flank = gb_record.seq[end_pos:end_pos+200]
#         else:
#             downstream_flank = gb_record.seq[end_pos:]
# #     For -ve sense strand
#     elif strand_sense == -1:
#         if len(gb_record.seq[:start_pos]) >= 200:
#             downstream_flank = gb_record.seq[start_pos-200:start_pos].reverse_complement()
#         else:
#             downstream_flank = gb_record.seq[:start_pos].reverse_complement()
        
#         if len(gb_record.seq[end_pos:]) >= 200:
#             upstream_flank = gb_record.seq[end_pos:end_pos+200].reverse_complement()
#         else:
#             upstream_flank = gb_record.seq[end_pos:].reverse_complement()
    
# #     print(f'\n{upstream_flank}\n\n{downstream_flank}')
# #     data.append((gene_name, str(start_pos), str(end_pos), 
# #                 str(strand_sense), str(gene_seq), str(upstream_flank), str(downstream_flank)))
#     data.append((gene_name, start_pos, end_pos, strand_sense, 
#                  str(gene_seq), str(upstream_flank), str(downstream_flank)))    
# #     break

In [134]:
# # Creating an empty dataframe
# df = pd.DataFrame(data, columns=['gene_name',
#                            'start_position',
#                            'end_position',
#                            'strand_sense', 
#                            'gene_seq', 
#                            'upstream_flank', 
#                            'downstream_flank'])

# df

,gene_name,start_position,end_position,strand_sense,gene_seq,upstream_flank,downstream_flank
0,dnaA,409,1750,1,ATGGAAAATATATTAGACCTGTGGAACCAAGCCCTTGCTCAAATCG...,ATAAGTTGTGGATTGATTTCACACAGCTTGTGTAGAAGGTTGTCCA...,CAGGACCGGGGATCAATCGGGGAAAGTGTGAATAACTTTTCGGAAG...
1,dnaN,1938,3075,1,ATGAAATTCACGATTCAAAAAGATCGTCTTGTTGAAAGTGTCCAAG...,CAGCTTAAATAGCAGGACCGGGGATCAATCGGGGAAAGTGTGAATA...,TCCGATACACTGCTGCCGACCCGTCGGCAGCTTTTCTATTCGGTAT...
2,rlbA,3205,3421,1,ATGGCAAATCCGATTTCAATTGATACAGAGATGATTACACTCGGAC...,ACCTTTCTTAATTCGCACGCCGAATGATGAAACGATTGTACAGCTT...,AGCGGGTGACACTGATTGTATATCCAGAACTTAGAACTGACATCTT...
3,recF,3436,4549,1,TTGTATATCCAGAACTTAGAACTGACATCTTACCGCAACTACGACC...,TGATTACACTCGGACAATTCTTAAAATTAGCCGATGTGATTCAGTC...,AGAAATGAGGTGAGCAATTGTATATTCATTTAGGTGATGACTTTGT...
4,remB,4566,4812,1,TTGTATATTCATTTAGGTGATGACTTTGTGGTTTCAACACGAGATA...,CCCATTTTACTATTGGATGATGTACTGAGTGAACTGGATGATTATC...,AAATTTTTTATCACGAATATATCGTTTAGAAAAGTGTAGGTGAATG...
...,...,...,...,...,...,...,...
4531,mnmE,4211509,4212889,-1,ATGGATACAATTGCGGCAATTTCCACACCGATGGGAGAAGGCGCGA...,TATCCTGTTGATAAAAAGGAAGTTCAGGAAATAAAATTAATTGTTA...,TTAAAGGAGGAACTAGAATCATGGGGTATGAAGCAGGCCAATACGA...
4532,jag,4213199,4213826,-1,GTGAGGAATGTGACTGCTGCAGGGCGAAATGTCGATGAAGCAGTAC...,AATGCGCAGCAAAATCCGCAAATGGCGATGATGCTTTGGATTATGC...,CATAAAACCGAAGTCCGATAAAAATTGGATTTCGGTTTTTTTGTAT...
4533,oxaAA,4213822,4214608,-1,ATGTTGTTGAAAAGGAGAATAGGGTTGCTATTAAGTATGGTTGGCG...,GTCTGCAGCATCTATTCAGAAAGTCTTCATTATATAAGAAATCTTC...,GGAATGTGACTGCTGCAGGGCGAAATGTCGATGAAGCAGTACAGTC...
4534,rnpA,4214752,4215103,-1,TTGAAGAAGCGAAATCGTTTAAAGAAAAATGAAGATTTTCAAAAAG...,AGTTTTAGCACGCCGTCGCCGCAAAGGCAGAAAAGTATTATCAGCT...,GCATCACCACTGCTTTCTTACAACTTTTAAAAGCGCTTCCTTTCTG...


In [135]:
# df[df.gene_name == 'unavailable']

,gene_name,start_position,end_position,strand_sense,gene_seq,upstream_flank,downstream_flank
313,unavailable,281674,281809,-1,TTGGCAAGAAGCATTATCTTGAAACAAGAACAGACTCATAATGGAA...,TCTTTATGTATTGCCGAAATGGAATAATATGATTGCAACTTTGGCT...,GAAAAGAACAAACCGAATGCCGCTAACGATAACGGGATATACACAA...
335,unavailable,300524,300656,-1,ATGACAAAGGAGGGCGGAGTTTCCAGAAAATCCGGTGAAAACCGTG...,TTTCTTCACAAAGGCGGCCCCTTTCATCCTTCTTAAGTTCCCTCCA...,ACCATTTGAGGAGGAAAAATGAATGTTTCAGAAAAAAACGTACGCT...
397,unavailable,369019,369217,1,TTGTCCAAGGATGAATGCCAGACGTACCAAGGAAACGCATGTGAAA...,ACTGAATATAACGGCTTTGGAATGCATGAATGATTTCACCTATAGT...,ACCATAATGGGGAGTCAAATGAATATTTACAAACCAGCCGGATTTT...
502,unavailable,490545,490749,-1,TTGATAAGAATAAGCATGCAAGGACAAATGCATCTTCCTTTGAAAC...,GAGTCGCGCCCGGCATTGCCATGATCGTGCTCAGTATCATTTCCCT...,CCGTTTCAAACTTTTCTTCAGTGACGGCATTTCAAATTTTTGGTTT...
544,unavailable,524359,524476,1,ATGTGGAATTATGAATACACAGCGATAGCTGATGTGGATGTGGGAA...,CAATCGGTAGAAGAAATATCGAATTCGCTGAAAATGTATACGACAT...,AAGGACAATTTCAAAATGGAACGGAAGGAACATTGCTTCTGCAAGG...
...,...,...,...,...,...,...,...
4104,unavailable,3804713,3804875,-1,ATGTTTTTGTCCTCGCTTCCTTCAAAACTCATTAAGACAATCCTTT...,GGGTGTGAGCCCGCTTTAACTTTTGCAGAAAAGCGTCAACTCTATC...,GATATTGACTATGAAAAGCGTGGTGTTTTTATGAAAGACGTATCTA...
4155,unavailable,3856781,3856937,-1,ATGCTTTTTACGGTTTCCAGAGAGGGAAGGACAGCTGAGATCTTCC...,TTGGTATGCTGGTGCCGGGTTTGCGTTTTTGATGATGGGAGCGGCA...,TCGGGAATTTGGGTGGAACCACGGATGATCAACACATTCGTCCCTT...
4251,unavailable,3951823,3951958,-1,ATGGAATGTTTCGAGCTGATTCATAATAAAGGAAAAGATACCTGGA...,TGTCCACTTGACAGCGGGTTTTTCATACAGCTGTTTCATTAATTGC...,TGAAATCAGGGCGAGAAGCGTATAATAAGAAAGAAAACGCCTTCTT...
4452,unavailable,4147418,4147547,-1,TTGCCAGGCTGTTTTGATATGCAAAAAAAACTGAACAGGGGTCACG...,GCTGAAGGAACTGGATGTCATCACGGAATTTGACGGATACAAAGTA...,CTGCAAATACTTCCGTTATTTGCATGGACTTTAATGGTTTGTACCG...


In [139]:
# del data
# data = []

In [ ]:
# # 'gene_seq' can be extracted using string splicing with added .reverse_complement() for -ve sense strands
# # Way 1:
# if .strand == 1:
#     gene_seq = gb_record.seq[gb_record.features[3].location.nofuzzy_start:gb_record.features[3].location.nofuzzy_end]
# elif .strand == -1:
#     gene_seq = gb_record.seq[gb_record.features[3].location.nofuzzy_start:gb_record.features[3].location.nofuzzy_end].reverse_complement()
# # Way 2:
# gene.extract(gb_record).seq

# ----------------------------------------------------------------------

# # For +ve sense strand
# for +ve sense strand, upstream_flank is obtained by 
# if len(gb_record.seq[:gb_record.features[3].location.nofuzzy_start] >= 200):
#     [gb_record.features[3].location.nofuzzy_start-200:gb_record.features[3].location.nofuzzy_start]
# else:
#     [:gb_record.features[3].location.nofuzzy_start]
    
# for +ve sense strand, downstream_flank is obtained by 
# if len(gb_record.seq[gb_record.features[3].location.nofuzzy_end:] >= 200):
#     [gb_record.features[3].location.nofuzzy_end:gb_record.features[3].location.nofuzzy_end+200]
# else:
#     [gb_record.features[3].location.nofuzzy_end:]
    
    
# # 'upstream_flank' and 'downstream_flank'; caution required for -ve sense strand
# # For -ve sense strand
# for -ve sense strand, upstream_flank is obtained by 
# if len(gb_record.seq[gb_record.features[3].location.nofuzzy_end:] >= 200):
#     [gb_record.features[3].location.nofuzzy_end:gb_record.features[3].location.nofuzzy_end+200].reverse_complement()
# else:
#     [gb_record.features[3].location.nofuzzy_end:].reverse_complement()
    
# for -ve sense strand, downstream_flank is obtained by 
# if len(gb_record.seq[:gb_record.features[3].location.nofuzzy_start] >= 200):
#     [gb_record.features[3].location.nofuzzy_start-200:gb_record.features[3].location.nofuzzy_start].reverse_complement()
# else:
#     [:gb_record.features[3].location.nofuzzy_start].reverse_complement()


In [77]:
# allgenes = (
#     feature
#     for feature in gb_record.features
#     if feature.type == 'gene'
# )

# for gene in allgenes:
#     if gene.strand == -1:
#         gene_name = gene.qualifiers['gene'][0]
#         start_pos = gene.location.nofuzzy_start
#     #     start_pos = gene.location.nofuzzy_start + 1 # if 1-based indexing required
#         end_pos = gene.location.nofuzzy_end
#         strand_sense = gene.strand

#         gene_seq = gene.extract(gb_record).seq
#         gene_seq2 = gb_record.seq[start_pos:end_pos]
#         gene_seq3 = gb_record.seq[start_pos:end_pos].reverse_complement()
#         print(gene_name, start_pos, end_pos, strand_sense)
#         print(f'\n{gene_seq}\n\n{gene_seq2}\n\n{gene_seq3}')
#         print(gene_seq == gene_seq2)
#         print(gene_seq == gene_seq3)
#         break

yaaC 14846 15794 -1

ATGACATATCATGAGTGGAAAGACTTAGCGCTATTTTACTCCGTTGAATCAACCCAGAAATTTCTAGAAAAGGTTTATATCTTAAACGGTATTAACGATGCGAAAAAAAACTCTTTTAAAAACAGTGAACGGTTTATTTATTTCTTAAAGCATGCCGAATCGTTTTATAAACAAGCCGCATATTCTCCCTTAGAAATCAAACCGATTCTTTTATTTTATGGAATGGCACAGCTTATCAAAGCATGCTTAATTACACGTGATCCTCATTATCCAAGCCATACATCAGTACTGGCACATGGCGTAACAACAAGAAAACGAAAGAAACAAAATTATTGTTTTAGCGACGATGAAGTGAAAATACAACGGAATGGCCTATGCGTACATTTTATGAAACATTTATTCGGACAATCGGATATAGTAGATGAACGGTATACCATGAAAAAATTATTAATGGCCATTCCAGAACTTAGCGATATTTTCTACTTTCAGCAGAAAGAACGATTTATGACAAAAGTTGAGAAGGATAAGAACGAAATTTTTGTACCGGAGGAAGTAGTAATCAATTACAAAATGTCTGATTCAAGGTTTGCTGAATACATGTCCCATCATTATCAATGGTCTTTTACAAAAAAAAATGAACACGGGCTGCTGTTCGAAATTTCACCACAAGACAAGGAGCCATGGACATCGACCAGCTTATTGTTTGACATGGAAAAGAACCAATATTATATCCCTTCTCAAAGGGAGCAATTTCTGAGGCTTCCCGAAATGACAATTCATTATTTGATTTTGTACAACGTTGGGATGATTGCAAGGTATGAAACAGAATGGTGGTATGAGTTATTAACACAGCATATAAGTGATGACTACGTGTTAATTCAGCAGTTTTTATTAGTATCAGAAAAGAAATTCCCTAAATACGCTTCACAGTTTCTTCTTCATTTTTAA

TTAAAAATGAAGAAGAAACTGTGAAGCGT

In [76]:
# allgenes = (
#     feature
#     for feature in gb_record.features
#     if feature.type == 'gene'
# )

# for gene in allgenes:
#     gene_name = gene.qualifiers['gene'][0]
#     start_pos = gene.location.nofuzzy_start
# #     start_pos = gene.location.nofuzzy_start + 1 # if 1-based indexing required
#     end_pos = gene.location.nofuzzy_end
#     strand_sense = gene.strand
    
#     gene_seq = gene.extract(gb_record).seq
#     gene_seq2 = gb_record.seq[start_pos:end_pos]
#     print(gene_name, start_pos, end_pos, strand_sense)
#     print(f'\n{gene_seq}\n\n{gene_seq2}')
#     print(gene_seq == gene_seq2)
#     break

dnaA 409 1750 1

ATGGAAAATATATTAGACCTGTGGAACCAAGCCCTTGCTCAAATCGAAAAAAAGTTGAGCAAACCGAGTTTTGAGACTTGGATGAAGTCAACCAAAGCCCACTCACTGCAAGGCGATACATTAACAATCACGGCTCCCAATGAATTTGCCAGAGACTGGCTGGAGTCCAGATACTTGCATCTGATTGCAGATACTATATATGAATTAACCGGGGAAGAATTGAGCATTAAGTTTGTCATTCCTCAAAATCAAGATGTTGAGGACTTTATGCCGAAACCGCAAGTCAAAAAAGCGGTCAAAGAAGATACATCTGATTTTCCTCAAAATATGCTCAATCCAAAATATACTTTTGATACTTTTGTCATCGGATCTGGAAACCGATTTGCACATGCTGCTTCCCTCGCAGTAGCGGAAGCGCCCGCGAAAGCTTACAACCCTTTATTTATCTATGGGGGCGTCGGCTTAGGGAAAACACACTTAATGCATGCGATCGGCCATTATGTAATAGATCATAATCCTTCTGCCAAAGTGGTTTATCTGTCTTCTGAGAAATTTACAAACGAATTCATCAACTCTATCCGAGATAATAAAGCCGTCGACTTCCGCAATCGCTATCGAAATGTTGATGTGCTTTTGATAGATGATATTCAATTTTTAGCGGGGAAAGAACAAACCCAGGAAGAATTTTTCCATACATTTAACACATTACACGAAGAAAGCAAACAAATCGTCATTTCAAGTGACCGGCCGCCAAAGGAAATTCCGACACTTGAAGACAGATTGCGCTCACGTTTTGAATGGGGACTTATTACAGATATCACACCGCCTGATCTAGAAACGAGAATTGCAATTTTAAGAAAAAAGGCCAAAGCAGAGGGCCTCGATATTCCGAACGAGGTTATGCTTTACATCGCGAATCAAATCGACAGCAATATTCGGGAACTCGAAGGAGCATTAATCAGAGTTGTCGCTTATTCATCTTT

### Loading the csvs

In [314]:
mapping_dict = {}

mapping_values_list = [i for i in range(-200, 0, 1)]
mapping_keys_list = [i for i in range(1, 201, 1)]

for i in range(len(mapping_keys_list)):
    mapping_dict[mapping_keys_list[i]] = mapping_values_list[i]
    
print(mapping_dict)

{1: -200, 2: -199, 3: -198, 4: -197, 5: -196, 6: -195, 7: -194, 8: -193, 9: -192, 10: -191, 11: -190, 12: -189, 13: -188, 14: -187, 15: -186, 16: -185, 17: -184, 18: -183, 19: -182, 20: -181, 21: -180, 22: -179, 23: -178, 24: -177, 25: -176, 26: -175, 27: -174, 28: -173, 29: -172, 30: -171, 31: -170, 32: -169, 33: -168, 34: -167, 35: -166, 36: -165, 37: -164, 38: -163, 39: -162, 40: -161, 41: -160, 42: -159, 43: -158, 44: -157, 45: -156, 46: -155, 47: -154, 48: -153, 49: -152, 50: -151, 51: -150, 52: -149, 53: -148, 54: -147, 55: -146, 56: -145, 57: -144, 58: -143, 59: -142, 60: -141, 61: -140, 62: -139, 63: -138, 64: -137, 65: -136, 66: -135, 67: -134, 68: -133, 69: -132, 70: -131, 71: -130, 72: -129, 73: -128, 74: -127, 75: -126, 76: -125, 77: -124, 78: -123, 79: -122, 80: -121, 81: -120, 82: -119, 83: -118, 84: -117, 85: -116, 86: -115, 87: -114, 88: -113, 89: -112, 90: -111, 91: -110, 92: -109, 93: -108, 94: -107, 95: -106, 96: -105, 97: -104, 98: -103, 99: -102, 100: -101, 101: -1

How to iterate rows of a dataframe: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas

In [317]:
def motif_analysis(df, file_name):
    """Input: dataframe loaded from csv file and its file name
    Output: dataframes for upstream- and downstream- flank's motif analysis
    """
#     pattern = re.compile(r'GAA[GA]')
    upstream_dict = {}
    downstream_dict = {}
    for key in keys_list:
        upstream_dict[key] = 0
        downstream_dict[key] = 0

    # make sure indexes pair with number of rows
    df = df.reset_index()
    for index, row in df.iterrows():
        # pd.isnull statements were added to handle the cases with no flank regions
        if not pd.isnull(row['upstream_flank_203']):
            for m in re.finditer(pattern, row['upstream_flank_203']):
                upstream_dict[m.start()+1] += 1
        if not pd.isnull(row['downstream_flank_203']):
            for m in re.finditer(pattern, row['downstream_flank_203']):
                downstream_dict[m.start()+1] += 1
#         if not pd.isnull(row['upstream_flank']):
#             for m in re.finditer(pattern, row['upstream_flank']):
#                 upstream_dict[m.start()+1] += 1
#         if not pd.isnull(row['downstream_flank']):
#             for m in re.finditer(pattern, row['downstream_flank']):
#                 downstream_dict[m.start()+1] += 1
    
    mapped_upstream_dict = {}
    for key in upstream_dict:
        mapped_upstream_dict[mapping_dict[key]] = upstream_dict[key]    
    
    df_up = pd.DataFrame.from_dict(mapped_upstream_dict, orient='index')
    df_down = pd.DataFrame.from_dict(downstream_dict, orient='index')
    df_up.to_csv('./csv_files_for_motifs/upstream_'+file_name)
    df_down.to_csv('./csv_files_for_motifs/downstream_'+file_name)
    del df_up, df_down
#         print(row['upstream_flank'], '\n\n' , row['downstream_flank'])


In [318]:
keys_list = [i for i in range(1, 201)]
pattern = re.compile(r'GAA[GA]')

# To do the flank calculations for all files
directory = os.fsencode('./csv_files/')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
#     print(filename)
    if filename.endswith('.csv'):
        print(filename)
        df_temp = pd.read_csv('./csv_files/'+filename)
        print(len(df_temp.index))
        motif_analysis(df_temp, filename)

bacillus_subtilis.csv
4536
streptococcus_pneumoniae.csv
2157
klebsiella_pneumoniae.csv
5404


In [294]:
# mapped_upstream_dict = {}

# for key in upstream_dict:
#     mapped_upstream_dict[mapping_dict[key]] = upstream_dict[key]

{1: -200, 2: -199, 3: -198, 4: -197, 5: -196, 6: -195, 7: -194, 8: -193, 9: -192, 10: -191, 11: -190, 12: -189, 13: -188, 14: -187, 15: -186, 16: -185, 17: -184, 18: -183, 19: -182, 20: -181, 21: -180, 22: -179, 23: -178, 24: -177, 25: -176, 26: -175, 27: -174, 28: -173, 29: -172, 30: -171, 31: -170, 32: -169, 33: -168, 34: -167, 35: -166, 36: -165, 37: -164, 38: -163, 39: -162, 40: -161, 41: -160, 42: -159, 43: -158, 44: -157, 45: -156, 46: -155, 47: -154, 48: -153, 49: -152, 50: -151, 51: -150, 52: -149, 53: -148, 54: -147, 55: -146, 56: -145, 57: -144, 58: -143, 59: -142, 60: -141, 61: -140, 62: -139, 63: -138, 64: -137, 65: -136, 66: -135, 67: -134, 68: -133, 69: -132, 70: -131, 71: -130, 72: -129, 73: -128, 74: -127, 75: -126, 76: -125, 77: -124, 78: -123, 79: -122, 80: -121, 81: -120, 82: -119, 83: -118, 84: -117, 85: -116, 86: -115, 87: -114, 88: -113, 89: -112, 90: -111, 91: -110, 92: -109, 93: -108, 94: -107, 95: -106, 96: -105, 97: -104, 98: -103, 99: -102, 100: -101, 101: -1

In [256]:
# # df_list = [df_bac, df_strep, df_kleb]
# df_list = [df_bac]
# keys_list = [i for i in range(1, 201)]
# pattern = re.compile(r'GAA[GA]')

# for df in df_list:
    
#     upstream_dict = {}
#     downstream_dict = {}
#     for key in keys_list:
#         upstream_dict[key] = 0
#         downstream_dict[key] = 0

#     df = df.reset_index()  # make sure indexes pair with number of rows
#     for index, row in df.iterrows():
#         for m in re.finditer(pattern, row['upstream_flank']):
#             upstream_dict[m.start()+1] += 1
#         for m in re.finditer(pattern, row['downstream_flank']):
#             downstream_dict[m.start()+1] += 1
        
# #         break
# #         print(row['upstream_flank'], '\n\n' , row['downstream_flank'])
        

In [222]:
# df_bac.iloc[0]['downstream_flank']

'CAGGACCGGGGATCAATCGGGGAAAGTGTGAATAACTTTTCGGAAGTCATACACAGTCTGTCCACATGTGGATAGGCTGTGTTTCCTGTCTTTTTCACAACTTATCCACAAATCCACAGGCCCTACTATTACTTCTACTATTTTTTATAAATATATATATTAATACATTATCCGTTAGGAGGATAAAAATGAAATTCACG'

In [210]:
# df_bac['upstream_flank'].str.findall('GAA[GA]')

0       [GAAG, GAAA, GAAA, GAAA, GAAA, GAAG]
1                               [GAAA, GAAG]
2                               [GAAA, GAAA]
3                   [GAAG, GAAG, GAAA, GAAG]
4                         [GAAA, GAAG, GAAG]
                        ...                 
4531                      [GAAG, GAAA, GAAG]
4532                            [GAAA, GAAA]
4533                [GAAA, GAAA, GAAA, GAAA]
4534                            [GAAA, GAAA]
4535                [GAAA, GAAA, GAAA, GAAG]
Name: upstream_flank, Length: 4536, dtype: object

In [214]:
# df_bac['upstream_flank'].str.count('GAA[GA]').sum()

16361

In [241]:
# df_bac['downstream_flank'].str.count('GAA[GA]')

0       3
1       1
2       4
3       4
4       5
       ..
4531    1
4532    4
4533    8
4534    4
4535    5
Name: downstream_flank, Length: 4536, dtype: int64

In [301]:
df_kleb.iloc[0]

gene_ID                                                      11849782
gene_name                                                 unavailable
start_position                                                    381
end_position                                                      822
strand_sense                                                       -1
gene_seq            ATGGCAGACATTACACTTATCAGCGGCAGCACCCTGGGCAGTGCGG...
upstream_flank      AGCAGGTACTTATCAACAAGATCCAAACAATCGATGAAATTCAGTC...
downstream_flank    GAATTGTAAAGATCGTCCGATCTTCTGTGGATAACCATGCTTAAAA...
Name: 0, dtype: object

In [302]:
df_kleb.iloc[0]['upstream_flank']

'AGCAGGTACTTATCAACAAGATCCAAACAATCGATGAAATTCAGTCCACTGAAACTCTGATCGTGCTGCAGAACCCGATCATGCGCACCATCAAGCCCTGATCGCTTTTTTTAATCCCACATTTTTCCACAGGTAGATCCCAGCTCGCTCACAGCGTACAATACGCCACTCTGAACAGTTGAGCGGGCGATCGGCGATCT'